In [1]:
import random, re
from collections import defaultdict
import itertools
import pycosat
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, BasicAer, IBMQ, execute
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit.circuit.library.standard_gates import ZGate

In [3]:
def tcnfgen(n,k,horn=1):
    cnf = []
    def unique(l,k):
        t = random.randint(1,n)
        while(t in l):
            t = random.randint(1,n)
        return t
    r = (lambda : random.randint(0,1))
    def r_to_sign(x):
        if r() == 1:
            return x
        else:
            return -x
    for i in range(k):
        x = unique([],n)
        y = unique([x],n)
        z = unique([x, y],n)
        if horn:
            cnf.append([x, -y,-z])
        else:
            cnf.append([r_to_sign(x), r_to_sign(y),r_to_sign(z)])
    return cnf

In [4]:
def create_CNF(n, k):
    new_line = []
    line = tcnfgen(n, k)
    for clause in line:
        new_line.append(sorted(clause, key = abs))
    return new_line

In [5]:
def prepare_clause(circuit, clause):
    for element in clause:
        if element<0:
            circuit.x(abs(element))
    
    return circuit

In [6]:
def or_operator(circuit, clause, counter):
    order_clause = [abs(x) for x in clause]
    for element in order_clause:
        circuit.x(element)
    circuit.mcx(order_clause, counter)
    circuit.x(counter)
    for element in order_clause:
        circuit.x(element)
    
    return circuit

In [7]:
def oracle(circuit, n, k, CNF):
    counter = n+1 
    for clause in CNF:
        prepare_clause(circuit, clause)
        or_operator(circuit, clause, counter)
        counter = counter + 1
        prepare_clause(circuit, clause)
    
    return circuit

In [8]:
def change_if_unsatisfied(circuit, n):
    r = random.randint(1,n+1)
    circuit.x(0)
    circuit.cx(0, r)
    circuit.x(0)
    circuit.reset(0)
    
    return circuit

In [9]:
def Quantum_Schoning(circuit, n, k, CNF):
    
    oracle(circuit, n, k, CNF)
    circuit.mcx(list(range(n+1, n+k+1)), 0)
    circuit.barrier()
    circuit.reset(list(range(n+1, n+k+1)))
    change_if_unsatisfied(circuit, n)
    circuit.barrier()
    
    return circuit

In [10]:
n = 4
k = 5
new_line = create_CNF(n, k)

In [11]:
for sol in pycosat.itersolve(new_line):
    print(sol)

[1, -2, -3, -4]
[1, -2, -3, 4]
[1, 2, 3, 4]
[1, 2, -3, -4]
[1, 2, -3, 4]
[-1, 2, -3, -4]
[-1, -2, -3, -4]
[-1, -2, -3, 4]
[-1, -2, 3, -4]


In [14]:
QSchoning_circuit = QuantumCircuit(n+k+1, n+k+1)

for i in range(1, n+1):
    QSchoning_circuit.h(i)

iterations = 7
for i in range(iterations):
    Quantum_Schoning(QSchoning_circuit, n, k, new_line)

QSchoning_circuit.measure(list(range(1,n+1)), list(range(1,n+1)))

job = execute(QSchoning_circuit, Aer.get_backend('qasm_simulator'), shots=10000)
#job = execute(mycircuit, backend= simulator_backend, shots=8192)
counts = job.result().get_counts(QSchoning_circuit)
# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome, "is observed", counts[outcome], "times")
print("\n")

0000000000 is observed 607 times
0001000000 is observed 2508 times
0000100000 is observed 614 times
0110100000 is observed 643 times
0010000000 is observed 1263 times
0111100000 is observed 2485 times
0100100000 is observed 658 times
0110000000 is observed 613 times
0100000000 is observed 609 times


